In [157]:
import pandas as pd
import numpy as np

In [158]:
labels = pd.read_csv('labels.csv',header=0)
print(list(labels.columns.values))
print("\n")
labels= labels[pd.notnull(labels['gene'])]
# df.value_counts()

    
    
data=pd.read_csv('data.csv',header=0)
print(list(data.columns.values))
data = data[pd.notnull(data['ORF'])]

['gene', 'TCA', 'Resp', 'Ribo', 'Proteas', 'Hist', 'HTH']


['ORF', 'NAME', 'alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']


In [159]:
data.head(5)

,ORF,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,...,cold 20,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g
1,YBR166C,TYR1 TYROSINE BIOSYNTHESIS PREPHENATE DEH...,0.33,-0.17,0.04,-0.07,-0.09,-0.12,-0.03,-0.20,...,0.08,0.39,-0.17,0.23,0.20,0.20,-0.17,-0.69,0.14,-0.27
3,YOR357C,GRD19 SECRETION GOLGI PROTEIN ...,-0.64,-0.38,-0.32,-0.29,-0.22,-0.01,-0.32,-0.27,...,0.36,-0.20,-0.34,-0.14,-0.09,0.06,-0.17,0.04,-0.97,-1.79
5,YLR292C,SEC72 SECRETION ER PROTEIN TRA...,-0.23,0.19,-0.36,0.14,-0.40,0.16,-0.09,-0.12,...,0.21,-0.29,-0.36,-0.10,-0.29,-0.18,-0.34,-0.47,-0.43,-1.06
7,YGL112C,TAF60 TRANSCRIPTION TFIID 60 KD SU...,-0.69,-0.89,-0.74,-0.56,-0.64,-0.18,-0.42,-0.34,...,-0.01,-0.29,0.31,0.21,0.07,0.18,-0.14,-0.20,-0.43,-1.51
9,YIL118W,RHO3 CYTOSKELETON GTP-BINDING PR...,0.04,0.01,-0.81,NaN,-0.30,0.49,0.08,0.19,...,0.28,-0.17,0.26,0.07,-0.17,NaN,-0.10,-0.23,-0.51,-1.40


In [160]:
labels.head(5)

,gene,TCA,Resp,Ribo,Proteas,Hist,HTH
1,YAL001C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,YAL002W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,YAL003W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,YAL005C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,YAL010C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [161]:
data=data.iloc[:,2:]
x= data.values #convert to numpy array
labels=labels.iloc[:,1:]
y=labels.values #convert to numpy array

y[y== -1] = 0 #one hot encoding

In [162]:
column_average=np.nanmean(x,axis=0)

In [163]:
#to fill missing values with average
for i in range(x.shape[1]):
    data.iloc[:,i] = data.iloc[:,i].fillna(column_average[i])

In [164]:
split=int(0.8*x.shape[0])
x_train=x[0:split,]
x_test=x[split+1:,]

y_train=y[0:split,]
y_test=y[split+1:,]

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(1973, 79)
(1973, 6)
(493, 79)
(493, 6)


In [182]:
def getLoss(w,x,y,lam):
    m = x.shape[0] #First we get the number of training examples
    scores = np.dot(x,w) #Then we compute raw class scores given our input and current weights
    prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
    loss = (-1 / m) * np.sum(y * np.log(prob)) + (lam/2)*np.sum(w*w) #We then find the loss of the probabilities
    grad = (-1 / m) * np.dot(x.T,(y - prob)) + lam*w #And compute the gradient for that loss
    return loss,grad

In [183]:
def loss_value(w,x,y):
    m = x.shape[0] #First we get the number of training examples
    scores = np.dot(x,w) #Then we compute raw class scores given our input and current weights
    prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
    loss = (-1 / m) * np.sum(y * np.log(prob)) + (lam/2)*np.sum(w*w) 
    return loss

In [184]:
def softmax(z):
    z -= np.max(z)
    sm = (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
    return sm

In [194]:
def getProbsAndPreds(someX):
    probs = softmax(np.dot(someX,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds

In [186]:
w = np.zeros([x_train.shape[1],y_train.shape[1]])
lam = 1
iterations = 1000
learningRate = 1e-5
losses = []
for i in range(0,iterations):
    loss,grad = getLoss(w,x_train,y_train,lam)
    losses.append(loss)
    w = w - (learningRate * grad)
print (loss)

0.1661505641934585


In [187]:
train_loss=loss_value(w,x_train,y_train)
print("Training loss value is:")
print(train_loss)

Training loss value is:
0.16615054434628682


In [189]:
test_loss=loss_value(w,x_test,y_test)
print("Testing loss value is:")
print(test_loss)

Testing loss value is:
0.17083154394339028


In [195]:
prob_train,pred_train=getProbsAndPreds(x_train) #probability matrix and the prediction
prob_test,pred_test=getProbsAndPreds(x_test)